In [1]:
# from data_load import Data
# load = Data()
# load.HOMES_FOR_SALE()
# load.INCOME()
# load.HOMICIDES()
# load.POPULATION()
# load.ZIP_COUNTY()

In [2]:
from Final_Data_Output import Final_Data as FD
FD = FD()
clean_df = FD.Merge_all()                
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067103 entries, 0 to 1067102
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   bed                 1067103 non-null  float64
 1   bath                1067103 non-null  float64
 2   house_size          1067103 non-null  float64
 3   acre_lot            1067103 non-null  float64
 4   zip_code            1067103 non-null  float64
 5   Household_AGI       1067103 non-null  float64
 6   Total_Pop           1067103 non-null  float64
 7   Homicides           1067103 non-null  float64
 8   bed_bath_ratio      1067103 non-null  float64
 9   economic_health     1067103 non-null  float64
 10  lot_to_house_ratio  1067103 non-null  float64
 11  log_price           1067103 non-null  float64
dtypes: float64(12)
memory usage: 97.7 MB


In [3]:
from model_testing import Testing_Models as TM
tm = TM(clean_df.drop(columns=['log_price']), clean_df['log_price'])
glmm_mod = tm.GLMM()

C:\Users\ej_la\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
from Client_choice import Client_Opt as CO
client = CO(clean_df, glmm_mod['model'], glmm_mod['scale_fit'])
result = client.Client_prediction()